# Eval WEAC

Initialize models, run over a resolution of 5cm with a standardized weak layer.


In [1]:
# Auto reload modules
%load_ext autoreload
%autoreload all

In [2]:
import os
from typing import List
import numpy as np
from numpy.linalg import LinAlgError
import pandas as pd
from pprint import pprint
import tqdm

from weac_2.analysis import Analyzer
from weac_2.core.system_model import SystemModel
from weac_2.components import ModelInput, Segment, ScenarioConfig, WeakLayer, Layer
from weac_2.utils.snowpilot_parser import SnowPilotParser, convert_to_mm, convert_to_deg

In [3]:
number_of_files = 5

# Process multiple files
file_paths = []
for directory in os.listdir("data/snowpits"):
    for file in os.listdir(f"data/snowpits/{directory}"):
        if file.endswith(".xml"):
            file_paths.append(f"data/snowpits/{directory}/{file}")

paths: List[str] = []
parsers: List[SnowPilotParser] = []

for file_path in file_paths[:number_of_files]:
    snowpilot_parser = SnowPilotParser(file_path)
    paths.append(file_path)
    parsers.append(snowpilot_parser)

print(f"\nFound {len(paths)} files")


Found 5 files


In [ ]:
# Extract data from all PST files
error_paths = {}
error_values = {}

spacing = 50 # mm

data_rows = []
for i, (file_path, parser) in tqdm.tqdm(
    enumerate(zip(paths, parsers)), total=len(paths)
):
    # setup spacing
    height_1 = parser.snowpit.snow_profile.hs[0] * convert_to_mm[parser.snowpit.snow_profile.hs[1]]
    height_2 = parser.snowpit.snow_profile.profile_depth[0] * convert_to_mm[parser.snowpit.snow_profile.profile_depth[1]]
    if height_1 > height_2:
        raise ValueError("Height 1 is greater than height 2")
    # space evenly and append the last height
    spacing_count = int(height_1 / spacing)
    spacing_end = (spacing_count-1) * spacing
    spacing_array = np.linspace(0, spacing_end, spacing_count).tolist()
    spacing_array = [int(x) for x in spacing_array]
    spacing_array.insert(-1, height_1)
    print(spacing_array)
    exit()
    for spacing in spacing_array:
        # Extract layers
        layers, density_method = parser.extract_layers()
    try:
        # Extract slope angle
        if parser.snowpit.core_info.location.slope_angle is None:
            phi = 0.0
        else:
            phi = (
                parser.snowpit.core_info.location.slope_angle[0]
                * convert_to_deg[parser.snowpit.core_info.location.slope_angle[1]]
            )
        _, layers_above = parser.extract_weak_layer_and_layers_above(
            parser.snowpit.core_info.location.depth_top[0] * convert_to_mm[parser.snowpit.core_info.location.depth_top[1]],
            layers
        )

        # Extract layers
        try:
            layers, density_method = parser.extract_layers()
        except Exception as e:
            raise e

        cut_length = pst.cut_length[0] * convert_to_mm[pst.cut_length[1]]
        column_length = (
            pst.column_length[0] * convert_to_mm[pst.column_length[1]]
        )
        segments = [
            Segment(length=cut_length, has_foundation=False, m=0.0),
            Segment(
                length=column_length - cut_length,
                has_foundation=True,
                m=0.0,
            ),
        ]
        scenario_config = ScenarioConfig(system_type="-vpst", phi=phi)
        model_input = ModelInput(
            weak_layer=weak_layer,
            layers=layers_above,
            scenario_config=scenario_config,
            segments=segments,
        )
        pst_system = SystemModel(model_input=model_input)
        pst_analyzer = Analyzer(pst_system)
        G, GIc, GIIc = pst_analyzer.differential_ERR(unit="J/m^2")

        data_rows.append(
            {
                "file_path": file_path,
                "pst_id": pst_id,
                "column_length": column_length,
                "cut_length": cut_length,
                "phi": phi,
                # Weak Layer properties
                "rho_wl": weak_layer.rho,
                "E_wl": weak_layer.E,
                "HH_wl": weak_layer.hand_hardness,
                "GT_wl": weak_layer.grain_type,
                "GS_wl": weak_layer.grain_size,
                # Simulation results
                "G": G,
                "GIc": GIc,
                "GIIc": GIIc,
            }
        )
    except Exception as e:
        error_id = f"{i}.{pst_id}"
        error_paths[error_id] = file_path
        error_values[error_id] = e
        overall_excluded_psts += 1

dataframe = pd.DataFrame(data_rows)
# pprint(error_values)
print(f"\nFound {len(pst_paths)} files with PST tests")
print(f"Found {amount_of_psts} PST tests")
print("Length of the dataframe: ", len(dataframe))
print(f"Amount of excluded PSTs: {overall_excluded_psts}")

print(f"\nFailed to extract layers: {failed_to_extract_layers}")
print(f"Failed to extract weak layer: {failed_to_extract_weak_layer}")
print(f"Slope angle is None: {slope_angle_is_None}")
print(f"Cut length exceeds column length: {cut_length_exceeds_column_length}")
print(
    f"Added Failure Types: {failed_to_extract_layers + slope_angle_is_None + cut_length_exceeds_column_length + failed_to_extract_weak_layer}"
)

  0%|          | 0/5 [00:00<?, ?it/s]

[0.0, 50.0, 100.0, 150.0, 200.0, 250.0, 300.0, 350.0, 400.0, 450.0, 500.0, 550.0, 600.0, 650.0, 700.0, 750.0, 800.0, 850.0, 900.0, 950.0, 1000.0, 1050.0, 1100.0, 1150.0, 1200.0, 1250.0, 1300.0, 1350.0, 1400.0, 1450.0, 1500.0, 1550.0, 1600.0, 1650.0, 1700.0, 1750.0, 1800.0, 1850.0, 1900.0, 1950.0, 2000.0, 2050.0, 2100.0, 2150.0, 2200.0, 2250.0, 2300.0, 2350.0, 2400.0, 2450.0, 2500.0, 2550.0, 2600.0, 2650.0, 2700.0, 2750.0, 2800.0, 2850.0, 2900.0, 3000.0, 2950.0]


NameError: name 'pst_id' is not defined

: 